# NX-MIMOSA Multi-Sensor Fusion## Radar + Doppler + Track-to-Track AssociationThis notebook demonstrates:1. Sensor bias estimation and correction2. Doppler integration for velocity observability3. Track-to-track association between two radars4. OOSM (out-of-sequence measurement) handling

In [ ]:
import numpy as npfrom nx_mimosa import (    MultiTargetTracker, SensorBiasEstimator,    doppler_measurement_matrix, compute_radial_velocity,    t2ta_associate, fuse_tracks, OOSMHandler,    KalmanFilter3D, make_cv3d_matrices,)

## 1. Sensor Bias Estimation

In [ ]:
np.random.seed(42)bias_est = SensorBiasEstimator(alpha=0.05, min_samples=20)# Simulate sensor with 30m range bias and 0.5° azimuth biasTRUE_RANGE_BIAS = 30.0TRUE_AZ_BIAS = np.radians(0.5)for i in range(100):    innovation = np.array([        TRUE_RANGE_BIAS + np.random.randn() * 10,        TRUE_AZ_BIAS + np.random.randn() * 0.002,        np.random.randn() * 0.005,    ])    S = np.diag([100, 4e-6, 25e-6])    bias_est.update(innovation, S)bias = bias_est.get_bias()is_biased, dims = bias_est.is_biased()print(f"Range bias:   {bias.range_bias:.1f}m (truth: {TRUE_RANGE_BIAS}m)")print(f"Azimuth bias: {np.degrees(bias.azimuth_bias):.2f}° (truth: {np.degrees(TRUE_AZ_BIAS):.2f}°)")print(f"Biased: {is_biased}, dimensions: {dims}")# Correct a measurementraw = np.array([50000.0, 0.5, 0.1])corrected = bias_est.correct_measurement(raw)print(f"\nRaw range:       {raw[0]:.1f}m")print(f"Corrected range: {corrected[0]:.1f}m (delta: {raw[0]-corrected[0]:.1f}m)")

## 2. Doppler Integration

In [ ]:
# Target flying east at 300 m/s, sensor at originstate = np.array([50000, 10000, 5000, 300, 50, -10], dtype=float)sensor = np.array([0, 0, 0], dtype=float)# Compute geometry-aware Doppler measurement matrixH_dop = doppler_measurement_matrix(state, sensor)v_radial = compute_radial_velocity(state, sensor)print(f"Target position: {state[:3].astype(int)}")print(f"Target velocity: {state[3:6].astype(int)} m/s")print(f"Radial velocity: {v_radial:.1f} m/s")print(f"\nDoppler H matrix (velocity row):")print(f"  vx weight: {H_dop[3,3]:.3f}")print(f"  vy weight: {H_dop[3,4]:.3f}")print(f"  vz weight: {H_dop[3,5]:.3f}")

## 3. Track-to-Track Association

In [ ]:
# Two radars tracking the same scenenp.random.seed(42)# Run two independent trackersradar_a = MultiTargetTracker(dt=1.0, r_std=50.0)radar_b = MultiTargetTracker(dt=1.0, r_std=80.0)  # Less accuratefor scan in range(40):    t = scan + 1    truth1 = np.array([200*t, 500, 5000])    truth2 = np.array([3000, 150*t, 8000])        dets_a = np.vstack([truth1 + np.random.randn(3)*40, truth2 + np.random.randn(3)*40])    dets_b = np.vstack([truth1 + np.random.randn(3)*70, truth2 + np.random.randn(3)*70])        radar_a.process_scan(dets_a)    radar_b.process_scan(dets_b)# Convert tracks for T2TAtracks_a = [{'id': t.track_id, 'x': t.filter.x, 'P': t.filter.P}            for t in radar_a.confirmed_tracks]tracks_b = [{'id': t.track_id, 'x': t.filter.x, 'P': t.filter.P}            for t in radar_b.confirmed_tracks]matched, ua, ub = t2ta_associate(tracks_a, tracks_b)print(f"Radar A tracks: {len(tracks_a)}")print(f"Radar B tracks: {len(tracks_b)}")print(f"Matched pairs:  {len(matched)}")for m in matched:    print(f"  A:{m.track_a_id} ↔ B:{m.track_b_id}  dist={m.distance:.1f}m  conf={m.confidence:.2f}")# Fuse matched tracksif matched:    i, j = matched[0].track_a_id, matched[0].track_b_id    ta = next(t for t in tracks_a if t['id'] == i)    tb = next(t for t in tracks_b if t['id'] == j)    x_f, P_f = fuse_tracks(ta, tb)    print(f"\nFused position: {x_f[:3].astype(int)}")    print(f"Radar A uncertainty: {np.sqrt(ta['P'][0,0]):.1f}m")    print(f"Radar B uncertainty: {np.sqrt(tb['P'][0,0]):.1f}m")    print(f"Fused uncertainty:   {np.sqrt(P_f[0,0]):.1f}m ← always lower!")